In [1]:
import numpy as np
import math
from random import randint

## helper functions 
synthetic division calculations

In [2]:
def eval_poly(poly, x):
    L = len(poly)
    deg = L-1
    results = 0
    for i in range(L):
        results += np.multiply(poly[i], np.power(x, deg-i))
    return results

def quotient(poly, k, deg_divisor):
    deg_poly = len(poly)-1
    deg_quotient = deg_poly - deg_divisor
    coeff = []
    for i in range(deg_quotient + 1):
        coeff.append(eval_poly(poly[:i+1], k))
    return coeff

#Q(x)=x-k quotient
#R : remainder
def synth_div(poly, k):
    #I don't remember why the 1 is here
    Q = quotient(poly, k, 1)
    R = eval_poly(poly, k)
#     print('quotient: ' + str(Q))
#     print('remainder: ' + str(R))
    return Q, R

## Newton's method for finding roots

    -Works for real and complex roots
    -criteria for stopping Newton's method is if remainder < 10^(-6)
    -Takes in ploynomials of arbitrary length

In [84]:
# In[ ]:

def round_if_close_to_int(root):
    abs_real = np.abs(root.real)
    abs_imag = np.abs(root.imag)

    if np.abs(abs_real - np.round(abs_real)) < 1e-6:
        root = np.round(root.real) + 1j*root.imag
                
    if np.abs(abs_imag - np.round(abs_imag)) < 1e-6:
        root =  root.real + 1j*np.round(root.imag)
    return root
    
def newtons_method_one_root(poly, x0):
    root = 0 + 0j
    i = 0
    while not root: #can be paralellized
        Q, R = synth_div(poly, x0)
        if np.abs(R) < 1e-11:
            root = x0
            root = round_if_close_to_int(root)
            
            if np.abs(root.real) < 1e-6:
                root = 0 + 1j*root.imag
            if np.abs(root.imag) < 1e-6:
                root = root.real + 1j*0
                
            break
        x0 = x0 - np.divide(R, eval_poly(Q, x0))
        i = i + 1
            
    return root, i, Q

def print_roots(roots, iterations_per_root):
    for i in range(len(roots)):
        print('root ' + str(i+1) + ': {0:.5f}'.format(roots[i]) + \
              '. Found in ' + str(iterations_per_root[i]) +' iterations.')
        
#a = synth_div([1, -10, 35, -50, 24], 4) # (x-1)(x-2)(x-3)(x-4)
def newtons_method_all_roots(poly):
    L = len(poly)
    deg = L-1
    deg_track_solve = deg
    roots = []
    iterations_per_root = []
    total_iter = 0
    j = 1
    print('==========================================')
    print('Newton''s method using synthetic division ')
    print('polynomial of degree ' + str(deg))
    print()

    while deg_track_solve: #can be paralellized
        throw_the_dice = np.random.uniform(low=np.min(poly), high=np.max(poly), size=None)
        throw_the_dice2 = np.random.uniform(low=np.min(poly), high=np.max(poly), size=None)
#         throw_the_dice = np.random.random()
#         throw_the_dice = .5
#         throw_the_dice2 = np.random.random()
        
        x0 = throw_the_dice + 1j*throw_the_dice2
        
#         print('x0 guess for root ' + str(j) + ': ' + str(x0))
        root, i, Q = newtons_method_one_root(poly, x0)
        if np.imag(root) == 0:
            root = np.real(root)
        roots.append(root), iterations_per_root.append(i)
        total_iter += i
        deg_track_solve -= 1
        poly = Q    
        j += 1 #keeps track of iterations within while loop
#         if i > 1e3:
#             break
    print()
    print_roots(roots, iterations_per_root)
    print()
    print('found in ' + str(total_iter) + ' total iteration(s)')
    print()
    
    return np.asarray(roots), iterations_per_root, total_iter

In [85]:
def poly_solve(polynomial):
#     root = np.zeros((len(polynomial)-1,1))
#     iterations_per_root
    if len(polynomial) < 3:
        if len(polynomial) == 2:
            if polynomial[1] is not 0:
                root = -polynomial[0]/polynomial[1]
#                 return -polynomial[0]/polynomial[1]
            iterations_per_root, total_iter = 0, 0
        else: #either len == 1 or 0*x (polynomial[1] == 0)
            print('it''s a constant.')
    elif len(polynomial) == 3:
        a, b, c = polynomial[0], polynomial[1], polynomial[2]
        delta = b**2-4*a*c
        x1, x2 = (-b + np.sqrt(delta))/(2*a), (-b - np.sqrt(delta))/(2*a)
        roots = np.asarray([x1, x2])
        print(roots)
        iterations_per_root, total_iter = 0, 0
    else:
        roots, iterations_per_root, total_iter = newtons_method_all_roots(poly)
    
    return roots, iterations_per_root, total_iter

## tests with polynomials
    
    results can be compared with:
    https://www.mathportal.org/calculators/solving-equations/polynomial-equation-solver.php

In [86]:
    # poly = np.array([2, 1, -4, 4])
# poly = np.array([1, 5/6, 1/6])
# poly = np.array([1, -10, 35, -50, 24])
# poly = np.array([2, 3, 2, -2, 4])
poly = np.array([3, 0, 0, 0, 0, 0, 0, -2, 4])
# Complex roots:
# 1. x = -1.251-i*1.107
# 2. x = -1.251+i*1.107
# 3. x = 0.501-i*0.683
# 4. x = 0.501+i*0.683

roots, iterations_per_root, total_iter = poly_solve(poly)
# roots, iterations_per_root, total_iter = newtons_import multiprocessing as mp
# print("Number of processors: ", mp.cpu_count())

Newtons method using synthetic division 
polynomial of degree 8


root 1: 0.90631+0.33639j. Found in 15 iterations.
root 2: -0.35888+1.00734j. Found in 10 iterations.
root 3: 0.45373-0.91674j. Found in 11 iterations.
root 4: -1.00116-0.43601j. Found in 15 iterations.
root 5: 0.90631-0.33639j. Found in 10 iterations.
root 6: 0.45373+0.91674j. Found in 7 iterations.
root 7: -1.00116+0.43601j. Found in 9 iterations.
root 8: -0.35888-1.00734j. Found in 1 iterations.

found in 78 total iteration(s)



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: ComplexWarning: Casting complex values to real discards the imaginary part
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: ComplexWarning: Casting complex values to real discards the imaginary part
